In [1]:
%load_ext autoreload
%autoreload 2

In [89]:
from data_utils import *
from train_graph_vae import create_qm9_dataset
import torch_geometric.transforms as T
from torch_geometric.datasets import QM9

include_hydrogen = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform_list = [
    SelectQM9TargetProperties(properties=["homo", "lumo"]),
    SelectQM9NodeFeatures(features=["atom_type"]),
]
if not include_hydrogen:
    transform_list.append(DropQM9Hydrogen())

max_num_nodes = 29 if include_hydrogen else 9
transform_list += [
    AddAdjacencyMatrix(max_num_nodes=max_num_nodes),
    AddNodeAttributeMatrix(max_num_nodes=max_num_nodes),
    AddEdgeAttributeMatrix(max_num_nodes=max_num_nodes),
]

pre_transform = T.Compose(transform_list)
transform = T.Compose([
    RandomPermutation(max_num_nodes=max_num_nodes),
    T.ToDevice(device=device)
])

# note: when the pre_filter or pre_transform is changed, delete the data/processed folder to update the dataset
dataset = QM9(root="./data", pre_transform=pre_transform, pre_filter=qm9_pre_filter, transform=transform)

train_dataset, val_dataset, test_dataset = create_qm9_data_split(dataset=dataset)


In [93]:
train_dataset[1]

Data(x=[8, 4], edge_index=[2, 16], edge_attr=[16, 4], y=[1, 2], pos=[15, 3], z=[15], smiles='[H]C#CC([H])([H])C(=O)N1C([H])([H])C1([H])[H]', name='gdb_15733', idx=[1], adj_triu_mat=[1, 45], node_mat=[1, 9, 4], edge_triu_mat=[1, 36, 4])

In [91]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)